In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/chess/games.csv')

In [ ]:
data

# Preprocessing

In [ ]:
data['winner'].unique()

In [ ]:
data.query("winner != 'draw'")

In [ ]:
moves = np.array(data.query("winner != 'draw'")['moves'])

In [ ]:
moves

In [ ]:
labels = np.array(data.query("winner != 'draw'")['winner'].apply(lambda x : 1 if x == 'white' else 0))

In [ ]:
moves.shape

In [ ]:
labels.shape

In [ ]:
np.max(data['turns'])

## Finding unique moves in all games

In [ ]:
all_moves = set()

for move_list in moves:
    for move in move_list.split(" "):
        if move not in all_moves:
            all_moves.add(move)
            
max_vocab = len(all_moves) 

## Length of longest sequence of moves

In [ ]:
max_len = 0

for move_list in moves:
    if len(move_list.split(" ")) > max_len:
        max_len = len(move_list.split(" "))
        
print("Max length : " + str(max_len))

## Creating input vectors

In [ ]:
print(max_vocab)
print(max_len)

In [ ]:
tokenizer = Tokenizer(num_words = max_vocab)
tokenizer.fit_on_texts(moves)

sequences = tokenizer.texts_to_sequences(moves)

word_index = tokenizer.word_index

model_inputs = pad_sequences(sequences, maxlen = max_len)

In [ ]:
model_inputs.shape

In [ ]:
labels.shape

# Model training

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(model_inputs, labels, train_size = 0.7, random_state = 24)

In [ ]:
embedding_dim = 512

inputs = tf.keras.Input(shape = max_len)

embedding = tf.keras.layers.Embedding(
    input_dim = max_vocab,
    output_dim = embedding_dim,
    input_length = max_len
)(inputs)

gru = tf.keras.layers.GRU(units = embedding_dim)(embedding)

outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(gru)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        tf.keras.metrics.AUC(name = 'auc')
    ]
)

batch_size = 32
epochs = 7

history = model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    batch_size = batch_size,
    epochs = epochs,
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

# Results

In [ ]:
fig = px.line(
    history.history,
    y = ['loss', 'val_loss'],
    labels = {'x' : "Epochs", 'y' : "Loss"},
    title = "Loss over time"
)

fig.show()

In [ ]:
fig = px.line(
    history.history,
    y = ['auc', 'val_auc'],
    labels = {'x' : "Epochs", 'y' : "AUC"},
    title = "AUC over time"
)

fig.show()

In [ ]:
model.evaluate(x_test, y_test)